In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import torch
import pickle
from genre.base_model import GENRE

# Entity Disambiguation

In [3]:
model_path = "/checkpoint/fabiopetroni/GENRE/home/GeNeRe/__GENRE/models/ed"
checkpoint_file = "checkpoint_aidayago.pt"
model = (
    GENRE.from_pretrained(model_path, checkpoint_file=checkpoint_file)
    .eval()
    .to("cuda:0")
)

/private/home/ndecao/.conda/envs/genre37/lib/python3.7/site-packages/hydra/_internal/hydra.py:71: UserWarning: 
@hydra.main(strict) flag is deprecated and will removed in the next version.
See https://hydra.cc/docs/next/upgrades/0.11_to_1.0/strict_mode_flag_deprecated
  warnings.warn(message=msg, category=UserWarning)


In [7]:
# loading trie from KILT titles -- takes 1-2 min
with open("/checkpoint/fabiopetroni/GENRE/home/GeNeRe/__GENRE/data/kilt/trie.pkl", "rb") as f:
    trie = pickle.load(f)

In [8]:
def prefix_allowed_tokens_fn(batch_id, sent):
    return trie.get(sent.tolist())

model.sample(
    [" [START_ENT] London [END_ENT] is the capital of the UK."],
    beam=5,
    max_len_b=15,
    prefix_allowed_tokens_fn=prefix_allowed_tokens_fn,
)

[[{'text': 'London', 'logprob': tensor(-0.0879, device='cuda:0')},
  {'text': 'City of London', 'logprob': tensor(-2.0315, device='cuda:0')},
  {'text': 'London, Ontario', 'logprob': tensor(-2.9399, device='cuda:0')},
  {'text': 'Los Angeles', 'logprob': tensor(-3.1010, device='cuda:0')},
  {'text': 'England', 'logprob': tensor(-4.2353, device='cuda:0')}]]

# ------ from this point on the code might not work: I am refactoring -----

# Document Retieval

In [ ]:
# loading model -- takes 5-20 sec
model_path = "models/kilt"
checkpoint_file = "checkpoint.pt"
model = (
    GENRE.from_pretrained(model_path, checkpoint_file=checkpoint_file)
    .eval()
    .to("cuda:0")
)

In [ ]:
# No need to re-load the trie from above
# also no need to re-define `prefix_allowed_tokens_fn`
model.sample(
    ["Which US nuclear reactor hada major accident in 1979?"],
    beam=10,
    max_len_b=15,
    prefix_allowed_tokens_fn=prefix_allowed_tokens_fn,
)

# End-to-End Entity Linking

In [ ]:
from genre.entity_linking_model import GeNeRe

In [ ]:
# loading model (there is some logging from the Kolitas code)
# it takes 4-5 min to load (I will optimize it when I'm back - it can be reduced to half of it with some tricks)
# I haven't refatored this yet so:
# i) code is a mess (in `genre.entity_linking_model`)
# ii) it is different class than the models above
model = GeNeRe(
    model_path='models/el',
    checkpoint_file='checkpoint_aidayago.pt',
    device="cuda:0",
)

In [ ]:
# there is some logging
sentence = "London is the capital of the UK."
spans = model.get_prediction(sentence)
spans

In [ ]:
model.get_markdown(sentence, spans)

# End-to-End Entity Linking v2

In [ ]:
from genre.entity_linking_model_v2 import GENREForEndToEndEntityLinking

In [ ]:
model = GENREForEndToEndEntityLinking.from_pretrained(
    'models/el',
    checkpoint_file='checkpoint_aidayago.pt',
    device="cuda:0",
    mention_trie_file="data/el_v2/mention_trie.pkl",
    candidates_dict_file="data/el_v2/candidates_dict.pkl",
)

In [ ]:
model.sample(
    [" London is the capital of the UK."],
    beam=6,
    max_len_b=1024,
)